In [192]:
import os
import urllib
import shutil
import requests
import asyncio
import aiohttp
import aiofiles
import json
import pyautogui as pag
import pyperclip as clip
from os.path import join
from binascii import a2b_base64
from time import sleep as sleep
from uuid import uuid1

In [193]:
# ===================================== CONFIGS ==========================================

# If there is a variable path, put a "???" to be replaced by the value in "VARIABLE_PATH"
BASE_URL = "https://www.google.com/search?&q=???&tbm=isch"

# Path of output
PATH = join("fotos")

# Ignores a variable that already has a subdirectory in the PATH
IGNORE_DUPLICATES = True

# Delete all temporary files once finished
DELETE_ON_FINISHED = False

# Use the name of the file from URL (only works if url has the name and extension, like google.com/image.jpg)
USE_URL_NAME = False

# ========================================================================================

In [194]:
# Images
poing = """
neoni
"""

# Create Bundles of images
boing = """
"""

In [195]:
# Treats the individuals
paths = [x.strip() for x in poing.split("\n") if x != ""]


if boing:
    # Treats the bundles
    pong = [x.strip() for x in boing.split("\n") if x != ""]
    pong.append("//")
    pong.reverse()

    bundles = []
    current = None
    for poth in pong:
        if poth == "//":
            if current != None:
                bundles.append(current)
            current = []
        else:
            current.append(poth)
else: 
    pong = []
    
# Variable Path
VARIABLE_PATH = paths + [x for x in pong if x != "//"]

# The bundles
VARIABLE_BUNDLES = bundles
len(VARIABLE_PATH), len(VARIABLE_BUNDLES)

(1, 0)

In [196]:
def script(path):
    script = ""
    with open(path, "r", encoding="utf-8") as f:
        script = f.read()

    return script

def ensure_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def assert_path(path) -> bool:
    is_path = os.path.isdir(path)
    if not is_path:
        os.mkdir(path)
    
    return is_path

def fetch(path, image_url):
    try:
        print(image_url)
        r = requests.get(image_url, stream = True)
        r.raw.decode_content = True

        name = ""
        if USE_URL_NAME:
            name = image_url.split("/")[-1]
        else:
            ext = r.headers['content-type'].split("/")[-1]
            name = f"{uuid1()}.{ext}"
            
        with open(join(path, name),'wb') as f:
            shutil.copyfileobj(r.raw, f)
    except:
        fetch(path, image_url)

def fetch_all(path, data):
    assert_path(path)

    for image_url in data:
        fetch(path, image_url)
        

# Fetch a single Image 
# https://stackoverflow.com/questions/35388332/how-to-download-images-with-aiohttp
async def async_fetch(session: aiohttp.ClientSession, url: str, path: str) -> str:
    async with session.get(url) as resp:
        if resp.status == 200:
            name = ""
            if USE_URL_NAME:
                name = url.split("/")[-1]
            else:
                ext = resp.headers['content-type'].split("/")[-1]
                name = f"{uuid1()}.{ext}"

            async with aiofiles.open(join(path, name), mode='wb') as f:
                await f.write(await resp.read())
            return ""
        else:
            return url

# Fetch all URLs
async def async_fetch_all(session: aiohttp.ClientSession, urls: str, path: str) -> list[str]:
    assert_path(path)
    tasks = []
    for url in urls:
        task = asyncio.create_task(async_fetch(session, url, path))
        tasks.append(task)
    
    # Return the unsuccessfull requests
    res = await asyncio.gather(*tasks)
    res = [url for url in res if url != ""]
    return res


def callback_google(data: list[str], path: str) -> list[str]:
    assert_path(path)
    passthrough = []
    for img in data:
        if img.strip().startswith("data:image/") or not img or (len(img) < 20 and len(img) > 300):
            try:
                ext = img.replace("data:image/", "").split(";")[0]
                save_path = join(path, f'{uuid1()}.{ext}')

                binary_data = a2b_base64(img.split(",")[-1])
                with open(save_path, 'wb') as f:
                    f.write(binary_data)
            except Exception as e:
                print("Error in dataurl: ", e)
        elif img.strip().startswith(("http://", "https://")):
            passthrough.append(img)

    return passthrough

CALLBACK = callback_google

In [197]:
var_dir = os.listdir(PATH)
length = len(VARIABLE_PATH)
done = 1
pag.PAUSE = 0.3

JSON_PATH = join(PATH, "jsons")
assert_path(JSON_PATH)

# Gives 3 seconds to focus on a browser window (use chrome, to be sure)
sleep(3)

for urls in VARIABLE_PATH: # VARIABLE_PATH:
    print(f"{urls} => {done} / {length}")

    if urls in var_dir or f"{urls}.json" in os.listdir(JSON_PATH):
        print("SKIPPED - Already has a subdirectory or json")
        done += 1
        continue 

    # Focus on the search bar and puts the URL
    pag.hotkey('ctrl', 'l', interval=0.3)
    clip.copy(BASE_URL.replace("???", str(urls)))
    pag.hotkey("ctrl", "v", interval=0.3)
    pag.press("enter")

    # Awaits for the site to load
    sleep(5)

    # Opens terminal
    pag.hotkey('ctrl', 'shift', 'j')

    # Focus on console
    pag.hotkey("ctrl", "'", interval=0.5)

    # Puts the script
    clip.copy(script("googleimages.js"))
    pag.hotkey("ctrl", "v", interval=0.5)
    pag.press("enter")

    # Focus on the window, for the clipboard script to work
    pag.hotkey("alt", "tab", interval=0.5)
    sleep(20)
    
    # Pastes the returning output
    data = clip.paste()
    data = data.split(",")
    
    with open(join(JSON_PATH, f"{urls}.json"), "w") as f:
        json.dump(data, f)
        
    # Sleep, because the program needs a rest too...
    sleep(3)
    done += 1

# Rick rolls, for some reason
pag.hotkey('ctrl', 'l', interval=0.3)
clip.copy("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
pag.hotkey("ctrl", "v", interval=0.3)
pag.press("enter")


neoni => 1 / 1


In [198]:
done = 1
length = len(VARIABLE_PATH)

# pog = VARIABLE_PATH[265:]
# length = len(pog) 
# for url in pog:

for url in VARIABLE_PATH:
    print(f"{url} => {done} / {length}")
    # Read the json, if it doesn't exists, continue the loop
    try:
        with open(join(JSON_PATH, f"{url}.json"), "r") as f:
            leftovers = json.load(f)
    except FileNotFoundError:
        done += 1
        continue

    if not leftovers:
        done += 1
        continue 

    # Treat the informations before processing
    leftovers = CALLBACK(leftovers, join(PATH, str(url)))

    # Looping until all the urls have been taken care of
    while leftovers:
        print(leftovers)
        # Do whatever with the scraped data
        try:
            async with aiohttp.ClientSession() as session:
                leftovers = await async_fetch_all(session, leftovers, join(PATH, str(url)))
                # # Handling unsuccessfull requests in an async environment is tough, so just let the sync version do that  
                # if leftovers:
                #     fetch_all(join(PATH, urls), leftovers)
        except aiohttp.InvalidURL:
            print("Invalid URL!")
            continue
        except TimeoutError:
            print("Oopsie :(")
    # Remove the json
    # os.remove(join(PATH, url, f"{url}.json"))
    done += 1

if DELETE_ON_FINISHED:
    shutil.rmtree(join(PATH, "jsons"))

neoni => 1 / 1
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
Error in dataurl:  Incorrect padding
['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR7Rk3zJv0Gaf7YycKNJknxgmbT5t0UmY3TVA&usqp=CAU', 'https://encrypted-tbn2.gstatic.com/faviconV2?url=https://weareneoni.com&client=VFE&size=16&type=FAVICON&fallback_opts=TYPE', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd

In [199]:
BUNDLES_DIR = join(PATH, "bundles")
ensure_dir(BUNDLES_DIR)

for codes in VARIABLE_BUNDLES:
    name = codes[0]
    ensure_dir(join(BUNDLES_DIR, name))
    for code in codes:
        # Create the file paths
        source = join(PATH, code)
        destination = join(BUNDLES_DIR, name, code)

        # Create folder and gather all files
        ensure_dir(destination)
        allfiles = os.listdir(source)
        
        # Iterate on all files to move them to destination folder
        for f in allfiles:
            os.rename(join(source, f), join(destination, f))

In [200]:
from winotify import Notification

Notification(app_id="Image Scrapper", title="Status", msg="O Scrapper terminou de baixar suas imagens!", duration="long").show()

ModuleNotFoundError: No module named 'winotify'